<a href="https://colab.research.google.com/github/buganart/mocogan/blob/master/mocogan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

GPU 0: Tesla T4 (UUID: GPU-3afb5118-6d8b-ee4d-5a63-c16ca2a82571)
We have 2 CPU cores.


In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

Mounted at /content/drive


In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

ok!


# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
#@markdown - The video database directory
video_db_dir = "/content/drive/My Drive/VIDEOS/SINGLE_FILE" #@param {type:"string"}
# experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
#@markdown - The output directory of generated videos 
out_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-mocogan" #@param {type:"string"}
#@markdown - set wandb run id of logged run to resume from there
resume_id = "" #@param {type:"string"}
#@markdown - set batch_size, default: 16
batch_size = 16 #@param {type:"integer"}
#@markdown - set num of iterations, default: 120000
n_iter = 120000 #@param {type:"integer"}
#@markdown - learning rate of the model (all components)
lr = 0.0002 #@param {type:"number"}
#@markdown - the height and width of input (resized) video
img_size = 96 #@param {type:"integer"}
#@markdown - the number of channel of input (resized) video
nc = 3 #@param {type:"integer"}

#@markdown ##model parameters
ndf = 64 #@param {type:"integer"}
ngf = 64 #@param {type:"integer"}
d_E = 10 #@param {type:"integer"}
hidden_size = 100 #@param {type:"integer"}
d_C = 50 #@param {type:"integer"}
d_M = 10 #@param {type:"integer"}

In [ ]:
#@title Clone mocogan repo

!git clone https://github.com/buganart/mocogan

Cloning into 'mocogan'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 122 (delta 4), reused 13 (delta 4), pack-reused 109
Receiving objects: 100% (122/122), 158.94 MiB | 37.25 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
#@title Install Dependencies

%cd /content/mocogan
# %pip install -r requirements.txt
%pip install sk-video
clear_on_success("Dependencies installed.")

Dependencies installed.


In [ ]:
#@title Copy video files to runtime

local_wav_dir = Path("/content/mocogan/raw_data")
!find "{video_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/
clear_on_success("All files copied to this runtime.")


All files copied to this runtime.


In [ ]:
#@title RESIZE

import glob
import os
current_path = "/content/mocogan/"
resized_path = os.path.join(current_path, 'resized_data')
files = glob.glob(os.path.join(current_path, 'raw_data/*.*'))

for i, file in enumerate(files):
    resized_filename = os.path.join(resized_path, str(i))
    !ffmpeg -i "{file}" -pix_fmt yuv420p -vf scale="{img_size}":"{img_size}" "{resized_filename}".mp4

# crop=96:96:42:24
clear_on_success("All files resized.")

All files resized.


In [ ]:
#@title TRAIN

!python train.py \
--out_dir "$out_dir" \
--resume_id "$resume_id" \
--batch_size "$batch_size" \
--n_iter "$n_iter" \
--lr "$lr" \
--img_size "$img_size" \
--nc "$nc" \
--ndf "$ndf" \
--ngf "$ngf" \
--d_E "$d_E" \
--hidden_size "$hidden_size" \
--d_C "$d_C" \
--d_M "$d_M" \ 
--mode_run True

/content/mocogan/models.py:147: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(params)
/content/mocogan/models.py:157: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(params, 0)
[100/120000] (0d 0h 1m 1s) Loss_Di: 0.4790 Loss_Dv: 1.1403 Loss_Gi: 3.3367 Loss_Gv: 3.1552 Di_real_mean 0.7616 Di_fake_mean 0.0798 Dv_real_mean 0.9336 Dv_fake_mean 0.5426
[200/120000] (0d 0h 2m 7s) Loss_Di: 1.9010 Loss_Dv: 0.9459 Loss_Gi: 5.0629 Loss_Gv: 3.5948 Di_real_mean 0.1303 Di_fake_mean 0.0009 Dv_real_mean 0.8820 Dv_fake_mean 0.4261
[300/120000] (0d 0h 3m 14s) Loss_Di: 0.5362 Loss_Dv: 0.5261 Loss_Gi: 2.9751 Loss_Gv: 3.4374 Di_real_mean 0.7672 Di_fake_mean 0.1018 Dv_real_mean 0.7971 Dv_fake_mean 0.1228
[400/120000] (0d 0h 4m 21s) Loss_Di: 0.8935 Loss_Dv: 1.0055 Loss_Gi: 0.5983 Loss_Gv: 5.2417 Di_real_mean 0.4176 Di_fake_mean 0.0118 Dv_real_mean 0.9902 Dv_fake_mean 0.3892
[500/120000] (0d 0h 5m 